In [ ]:
# Imports
import pandas as pd
import numpy as np
from google.colab import files
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


# Import and clean 8th grade math scores for each year

In [ ]:
# Import csv files downloaded from State of Michigan website
MIData2015_2016 = pd.read_csv('/drive/My Drive/SIADS_697_698_Capstone/4_Colabs/2_Michigan_Academic_Data_from_MI_School_Data/1_Raw_Data/Grades_3_8_2015_2016.csv') 
MIData2016_2017 = pd.read_csv('/drive/My Drive/SIADS_697_698_Capstone/4_Colabs/2_Michigan_Academic_Data_from_MI_School_Data/1_Raw_Data/Grades_3_8_2016_2017.csv') 
MIData2017_2018 = pd.read_csv('/drive/My Drive/SIADS_697_698_Capstone/4_Colabs/2_Michigan_Academic_Data_from_MI_School_Data/1_Raw_Data/Grades_3_8_2017_2018.csv') 
MIData2018_2019 = pd.read_csv('/drive/My Drive/SIADS_697_698_Capstone/4_Colabs/2_Michigan_Academic_Data_from_MI_School_Data/1_Raw_Data/Grades_3_8_2018_2019.csv') 
MIData2020_2021 = pd.read_csv('/drive/My Drive/SIADS_697_698_Capstone/4_Colabs/2_Michigan_Academic_Data_from_MI_School_Data/1_Raw_Data/Grades_3_8_2020_2021.csv') 

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2,9,19,20,21,22,23,24,25,29,30,31,32,33,34,35,38,39,40,41,42,43,44,45) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2,9,19,20,21,22,23,24,25,29,30,31,32,33,34,35,38,40,41,42,43,44,45) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# Function to clean the dataframe for each school year's data
def clean_dataframe(df, marker, year):
  df = df.rename(columns=lambda x: x.strip())
  df.columns = df.columns.str.lower()  
  df = df[df['gradecontenttested'] == 8]
  df = df[['schoolyear', 'testtype', 'isdcode', 'isdname',
       'districtcode', 'districtname', 'buildingcode', 'buildingname',
       'countycode', 'countyname', 'entitytype', 'schoollevel', 'locale',
       'mistem_name', 'mistem_code', 'gradecontenttested', 'subject', 'reportcategory',
       'numberassessed', 'percentmet']]
  df = df[df['subject'].str.contains('Mathematics')]
  df = df[df['reportcategory'].str.contains('All Students')]
  df = df[df['buildingname'].str.contains('Middle')]

  # Filter for test type
  if marker == 1:
    df = df[df['testtype'].str.contains('M-STEP')] # For years through 2017-2018 school year
  else:
    df = df[df['testtype'].str.contains('PSAT')] # For years after 2017-2018
  
  # Replace <=10%, etc. with integers
  df.loc[df.percentmet == '>=80%', 'percentmet'] = 90
  df.loc[df.percentmet == '>=90%', 'percentmet'] = 95
  df.loc[df.percentmet == '<=5%', 'percentmet'] = 2.5
  df.loc[df.percentmet == '<=10%', 'percentmet'] = 5
  df.loc[df.percentmet == '<=20%', 'percentmet'] = 10
  df.loc[df.percentmet == '<=50%', 'percentmet'] = np.NaN
  df.loc[df.percentmet == '*', 'percentmet'] = np.NaN

  # Trim dataframe down to essentials
  df = df[['schoolyear', 'districtcode', 'districtname', 'buildingcode', 'buildingname',
       'countycode', 'countyname', 'percentmet']]
      
  # Cast 'percentmet' column as floats
  df['percentmet'] = df['percentmet'].astype(float) 

  # Groupby school district using mean of middle schools
  df = df.groupby(['districtcode'], as_index=False).agg({'schoolyear':'first', 'districtname':'first', 'percentmet':([np.mean])})
  df.columns = df.columns.droplevel(1)

  # Name percentmet with year suffix
  df = df.rename(columns = {'percentmet' : '8th_math_percentmet'})

  # Drop rows with NaN
  df = df.dropna()

  # Cast 'districtcode' as string
  df['districtcode'] = df['districtcode'].astype(str)
  df = df.sort_values('districtcode')

  return df

In [ ]:
# Clean dataframe. Marker of 1 indicates M-STEP, 2 indicates PSAT
MIData2015_2016c = clean_dataframe(MIData2015_2016, 1, '2015_2016')
MIData2016_2017c = clean_dataframe(MIData2016_2017, 1, '2016_2017')
MIData2017_2018c = clean_dataframe(MIData2017_2018, 1, '2017_2018')
MIData2018_2019c = clean_dataframe(MIData2018_2019, 2, '2018_2019')
MIData2020_2021c = clean_dataframe(MIData2020_2021, 2, '2020_2021')

In [ ]:
len(MIData2015_2016c), len(MIData2016_2017c), len(MIData2017_2018c), len(MIData2018_2019c), len(MIData2020_2021c)

(366, 367, 358, 359, 350)

In [ ]:
# Clean 'schoolyear' column
MIData2015_2016c['schoolyear'] = '2015-2016'
MIData2016_2017c['schoolyear'] = '2016-2017'
MIData2017_2018c['schoolyear'] = '2017-2018'
MIData2018_2019c['schoolyear'] = '2018-2019'
MIData2020_2021c['schoolyear'] = '2020-2021'

In [ ]:
# Identify schools in each year
MIData2015_2016c_School_List = MIData2015_2016c['districtcode'].tolist()
MIData2016_2017c_School_List = MIData2016_2017c['districtcode'].tolist()
MIData2017_2018c_School_List = MIData2017_2018c['districtcode'].tolist()
MIData2018_2019c_School_List = MIData2018_2019c['districtcode'].tolist()
MIData2020_2021c_School_List = MIData2020_2021c['districtcode'].tolist()

# Create list of schools in every year
five_four = list(set(MIData2020_2021c_School_List).intersection(set(MIData2018_2019c_School_List)))
five_four_three = list(set(five_four).intersection(set(MIData2017_2018c_School_List)))
five_four_three_two = list(set(five_four_three).intersection(set(MIData2016_2017c_School_List)))
five_four_three_two_one = list(set(five_four_three_two).intersection(set(MIData2015_2016c_School_List)))

# Filter dataframes using list of schools present in every year
MIData2015_2016c = MIData2015_2016c[MIData2015_2016c['districtcode'].isin(five_four_three_two_one)]
MIData2016_2017c = MIData2016_2017c[MIData2016_2017c['districtcode'].isin(five_four_three_two_one)]
MIData2017_2018c = MIData2017_2018c[MIData2017_2018c['districtcode'].isin(five_four_three_two_one)]
MIData2018_2019c = MIData2018_2019c[MIData2018_2019c['districtcode'].isin(five_four_three_two_one)]
MIData2020_2021c = MIData2020_2021c[MIData2020_2021c['districtcode'].isin(five_four_three_two_one)]

# Make sure 12 schools with cell phone bans are in master list of 413 schools with scores every year
Schools_With_Bans = ['34010', '41110', '82055', '82100']
MIData2020_2021c.loc[MIData2020_2021c['districtcode'].isin(Schools_With_Bans)]
type(MIData2020_2021c['districtcode'][0])

str

In [ ]:
# # Create final dataframe with 'percentmet' for each year
# MIData_Scores = MIData2015_2016c.copy()
# MIData_Scores['percentmet2016_2017'] = MIData2016_2017c['percentmet2016_2017'].values
# MIData_Scores['percentmet2017_2018'] = MIData2017_2018c['percentmet2017_2018'].values
# MIData_Scores['percentmet2018_2019'] = MIData2018_2019c['percentmet2018_2019'].values
# MIData_Scores['percentmet2020_2021'] = MIData2020_2021c['percentmet2020_2021'].values

# # Drop 'schoolyear' column
# MIData_Scores=MIData_Scores.drop(['schoolyear'], axis=1)

# # MIData_Scores = MIData_Scores[['buildingcode', 'percentmet2015_2016', 'percentmet2016_2017', 'percentmet2017_2018', 'percentmet2018_2019', 'percentmet2020_2021']]
# # MIData_Scores = MIData_Scores.T
# # MIData_Scores.columns = MIData_Scores.iloc[0]
# # MIData_Scores = MIData_Scores.drop(['buildingcode'])
# MIData_Scores

# All Other Metrics

In [ ]:
# Filter dataframe to only include schools in the MIData_Scores dataframe (341 schools)
# df1 is the attendance, etc dataframe. df2 is the currated MIData_Scores dataframe

def filter_schools(df1, df2):
  
  # setting the base for the left semi-join in python
  semi = df1.merge(df2, on = 'districtcode')
  df1['districtcode'].isin(df2['districtcode'])
  semi = df1.merge(df2, on = 'districtcode')
  
  # our left semi join
  new_semi=df1[df1['districtcode'].isin(semi['districtcode'])]
  return pd.DataFrame(new_semi)

In [ ]:
# Import csv files downloaded from State of Michigan website
Attendance_2015_2016 = pd.read_csv('/drive/My Drive/SIADS_697_698_Capstone/4_Colabs/2_Michigan_Academic_Data_from_MI_School_Data/1_Raw_Data/2015_2016_Attendance.csv') 
Attendance_2016_2017 = pd.read_csv('/drive/My Drive/SIADS_697_698_Capstone/4_Colabs/2_Michigan_Academic_Data_from_MI_School_Data/1_Raw_Data/2016_2017_Attendance.csv') 
Attendance_2017_2018 = pd.read_csv('/drive/My Drive/SIADS_697_698_Capstone/4_Colabs/2_Michigan_Academic_Data_from_MI_School_Data/1_Raw_Data/2017_2018_Attendance.csv') 
Attendance_2018_2019 = pd.read_csv('/drive/My Drive/SIADS_697_698_Capstone/4_Colabs/2_Michigan_Academic_Data_from_MI_School_Data/1_Raw_Data/2018_2019_Attendance.csv') 
Attendance_2020_2021 = pd.read_csv('/drive/My Drive/SIADS_697_698_Capstone/4_Colabs/2_Michigan_Academic_Data_from_MI_School_Data/1_Raw_Data/2020_2021_Attendance.csv') 

Ed_Effective_2015_2016 = pd.read_csv('/drive/My Drive/SIADS_697_698_Capstone/4_Colabs/2_Michigan_Academic_Data_from_MI_School_Data/1_Raw_Data/2015_2016_Educator_Effectiveness.csv') 
Ed_Effective_2016_2017 = pd.read_csv('/drive/My Drive/SIADS_697_698_Capstone/4_Colabs/2_Michigan_Academic_Data_from_MI_School_Data/1_Raw_Data/2016_2017_Educator_Effectiveness.csv') 
Ed_Effective_2017_2018 = pd.read_csv('/drive/My Drive/SIADS_697_698_Capstone/4_Colabs/2_Michigan_Academic_Data_from_MI_School_Data/1_Raw_Data/2017_2018_Educator_Effectiveness.csv') 
Ed_Effective_2018_2019 = pd.read_csv('/drive/My Drive/SIADS_697_698_Capstone/4_Colabs/2_Michigan_Academic_Data_from_MI_School_Data/1_Raw_Data/2018_2019_Educator_Effectiveness.csv') 
Ed_Effective_2020_2021 = pd.read_csv('/drive/My Drive/SIADS_697_698_Capstone/4_Colabs/2_Michigan_Academic_Data_from_MI_School_Data/1_Raw_Data/2020_2021_Educator_Effectiveness.csv') 

Finance_2015_2016 = pd.read_csv('/drive/My Drive/SIADS_697_698_Capstone/4_Colabs/2_Michigan_Academic_Data_from_MI_School_Data/1_Raw_Data/2015_2016_Financial_Reports.csv') 
# Fix Detroit
df1 = pd.DataFrame([[np.nan] * len(Finance_2015_2016.columns)], columns=Finance_2015_2016.columns)
Finance_2015_2016 = df1.append(Finance_2015_2016, ignore_index=True)
Finance_2015_2016.at[0, 'District Code'] = '82015'
Finance_2015_2016.at[0, 'District Name'] = 'Detroit Public Schools Community District'

Finance_2016_2017 = pd.read_csv('/drive/My Drive/SIADS_697_698_Capstone/4_Colabs/2_Michigan_Academic_Data_from_MI_School_Data/1_Raw_Data/2016_2017_Financial_Reports.csv') 
Finance_2017_2018 = pd.read_csv('/drive/My Drive/SIADS_697_698_Capstone/4_Colabs/2_Michigan_Academic_Data_from_MI_School_Data/1_Raw_Data/2017_2018_Financial_Reports.csv') 
Finance_2018_2019 = pd.read_csv('/drive/My Drive/SIADS_697_698_Capstone/4_Colabs/2_Michigan_Academic_Data_from_MI_School_Data/1_Raw_Data/2018_2019_Financial_Reports.csv') 
Finance_2020_2021 = pd.read_csv('/drive/My Drive/SIADS_697_698_Capstone/4_Colabs/2_Michigan_Academic_Data_from_MI_School_Data/1_Raw_Data/2020_2021_Financial_Reports.csv') 

Staff_2015_2016 = pd.read_csv('/drive/My Drive/SIADS_697_698_Capstone/4_Colabs/2_Michigan_Academic_Data_from_MI_School_Data/1_Raw_Data/2015_2016_Staffing_Count.csv') 
Staff_2016_2017 = pd.read_csv('/drive/My Drive/SIADS_697_698_Capstone/4_Colabs/2_Michigan_Academic_Data_from_MI_School_Data/1_Raw_Data/2016_2017_Staffing_Count.csv') 
Staff_2017_2018 = pd.read_csv('/drive/My Drive/SIADS_697_698_Capstone/4_Colabs/2_Michigan_Academic_Data_from_MI_School_Data/1_Raw_Data/2017_2018_Staffing_Count.csv') 
Staff_2018_2019 = pd.read_csv('/drive/My Drive/SIADS_697_698_Capstone/4_Colabs/2_Michigan_Academic_Data_from_MI_School_Data/1_Raw_Data/2018_2019_Staffing_Count.csv') 
Staff_2020_2021 = pd.read_csv('/drive/My Drive/SIADS_697_698_Capstone/4_Colabs/2_Michigan_Academic_Data_from_MI_School_Data/1_Raw_Data/2020_2021_Staffing_Count.csv') 

Student_2015_2016 = pd.read_csv('/drive/My Drive/SIADS_697_698_Capstone/4_Colabs/2_Michigan_Academic_Data_from_MI_School_Data/1_Raw_Data/2015_2016_Student_Count.csv') 
Student_2016_2017 = pd.read_csv('/drive/My Drive/SIADS_697_698_Capstone/4_Colabs/2_Michigan_Academic_Data_from_MI_School_Data/1_Raw_Data/2016_2017_Student_Count.csv') 
Student_2017_2018 = pd.read_csv('/drive/My Drive/SIADS_697_698_Capstone/4_Colabs/2_Michigan_Academic_Data_from_MI_School_Data/1_Raw_Data/2017_2018_Student_Count.csv') 
Student_2018_2019 = pd.read_csv('/drive/My Drive/SIADS_697_698_Capstone/4_Colabs/2_Michigan_Academic_Data_from_MI_School_Data/1_Raw_Data/2018_2019_Student_Count.csv') 
Student_2020_2021 = pd.read_csv('/drive/My Drive/SIADS_697_698_Capstone/4_Colabs/2_Michigan_Academic_Data_from_MI_School_Data/1_Raw_Data/2020_2021_Student_Count.csv') 

# Attendance

In [ ]:
def clean_attendance(df, year):
  df = df.rename(columns=lambda x: x.strip())
  df.columns = df.columns.str.lower()  
  df = df.loc[df['buildingname'] == 'All Buildings']
  df = df.loc[df['reportcategory'] == 'Grade']
  df = df.loc[df['reportsubgroup'] == 'All']
  df['schoolyear'] = year
  df = df[['districtcode', 'ar_allstudents']] # AR_AllStudents is attendance rate all students
  df = df.rename(columns = {'ar_allstudents' : 'attendance'})
  df['districtcode'] = df['districtcode'].astype(str)
  df = df.sort_values('districtcode')

  # df = df.loc[df['districtname'] == 'Forest Hills Public Schools']
  # df = df.loc[df['districtcode'] == '82015']
  
  return df

In [ ]:
Attendance_2015_2016c = clean_attendance(Attendance_2015_2016, '2015-2016')
Attendance_2016_2017c = clean_attendance(Attendance_2016_2017, '2016-2017')
Attendance_2017_2018c = clean_attendance(Attendance_2017_2018, '2017-2018')
Attendance_2018_2019c = clean_attendance(Attendance_2018_2019, '2018-2019')
Attendance_2020_2021c = clean_attendance(Attendance_2020_2021, '2020-2021')

Attendance_2015_2016cc = filter_schools(Attendance_2015_2016c, MIData2020_2021c)
Attendance_2016_2017cc = filter_schools(Attendance_2016_2017c, MIData2020_2021c)
Attendance_2017_2018cc = filter_schools(Attendance_2017_2018c, MIData2020_2021c)
Attendance_2018_2019cc = filter_schools(Attendance_2018_2019c, MIData2020_2021c)
Attendance_2020_2021cc = filter_schools(Attendance_2020_2021c, MIData2020_2021c)

# Student Count

In [ ]:
    
def clean_student(df, year):
  df = df.rename(columns=lambda x: x.strip())
  df.columns = df.columns.str.lower()  
  df = df.loc[df['buildingname'] == 'All Buildings']
  # df = df.loc[df['reportcategory'] == 'Grade']
  # df = df.loc[df['reportsubgroup'] == 'All']
  df['schoolyear'] = year
  df = df[['districtcode', 'american_indian_enrollment', 'asian_enrollment',
      'english_language_learners_enrollment', 'early_middle_college_enrollment']]

  # Replace <=10%, etc. with integers
  df.loc[df.english_language_learners_enrollment == '<10', 'english_language_learners_enrollment'] = 5
  df['districtcode'] = df['districtcode'].astype('Int64')
  df['districtcode'] = df['districtcode'].astype(str)
  df = df.sort_values('districtcode')

  # df = df.loc[df['districtname'] == 'Forest Hills Public Schools']
  return df

In [ ]:

Student_2015_2016c = clean_student(Student_2015_2016, '2015-2016')
Student_2016_2017c = clean_student(Student_2016_2017, '2016-2017')
Student_2017_2018c = clean_student(Student_2017_2018, '2017-2018')
Student_2018_2019c = clean_student(Student_2018_2019, '2018-2019')
Student_2020_2021c = clean_student(Student_2020_2021, '2020-2021')

Student_2015_2016cc = filter_schools(Student_2015_2016c, MIData2020_2021c)
Student_2016_2017cc = filter_schools(Student_2016_2017c, MIData2020_2021c)
Student_2017_2018cc = filter_schools(Student_2017_2018c, MIData2020_2021c)
Student_2018_2019cc = filter_schools(Student_2018_2019c, MIData2020_2021c)
Student_2020_2021cc = filter_schools(Student_2020_2021c, MIData2020_2021c)

# Educator Effectiveness:  Admin Ineffective, Admin Minimally Effective, Teacher Minimally Effective

In [ ]:
    
def clean_admin_ineffective(df, year):
  df = df.rename(columns=lambda x: x.strip())
  df.columns = df.columns.str.lower()  
  # df = df.loc[df['buildingname'] == 'All Buildings']
  df = df.loc[df['assignment_type'] == 'Administrator']
  df = df.loc[df['effectiveness_category'] == 'Ineffective']
  df['schoolyear'] = year
  df = df.rename(columns={'district code': 'districtcode', 'district name': 'districtname', 'percentage':'admin_ineffective_percentage'})
  df = df[['districtcode', 'admin_ineffective_percentage']]
  df['districtcode'] = df['districtcode'].astype(str)

  # df = df.loc[df['districtname'] == 'Forest Hills Public Schools']
  return df

In [ ]:
Admin_ineffective_2015_2016c = clean_admin_ineffective(Ed_Effective_2015_2016, '2015-2016')
Admin_ineffective_2016_2017c = clean_admin_ineffective(Ed_Effective_2016_2017, '2016-2017')
Admin_ineffective_2017_2018c = clean_admin_ineffective(Ed_Effective_2017_2018, '2017-2018')
Admin_ineffective_2018_2019c = clean_admin_ineffective(Ed_Effective_2018_2019, '2018-2019')
Admin_ineffective_2020_2021c = clean_admin_ineffective(Ed_Effective_2020_2021, '2020-2021')

Admin_ineffective_2015_2016cc = filter_schools(Admin_ineffective_2015_2016c, MIData2020_2021c)
Admin_ineffective_2016_2017cc = filter_schools(Admin_ineffective_2016_2017c, MIData2020_2021c)
Admin_ineffective_2017_2018cc = filter_schools(Admin_ineffective_2017_2018c, MIData2020_2021c)
Admin_ineffective_2018_2019cc = filter_schools(Admin_ineffective_2018_2019c, MIData2020_2021c)
Admin_ineffective_2020_2021cc = filter_schools(Admin_ineffective_2020_2021c, MIData2020_2021c)

In [ ]:
def clean_admin_min_effective(df, year):
  df = df.rename(columns=lambda x: x.strip())
  df.columns = df.columns.str.lower()  
  df = df.loc[df['assignment_type'] == 'Administrator']
  df = df.loc[df['effectiveness_category'] == 'MinimallyEffective']
  df['schoolyear'] = year
  df = df.rename(columns={'district code': 'districtcode', 'district name': 'districtname', 'percentage':'admin_min_effective_percentage'})
  df = df[['districtcode', 'admin_min_effective_percentage']]
  df['districtcode'] = df['districtcode'].astype(str)

  # df = df.loc[df['districtname'] == 'Forest Hills Public Schools']
  return df

In [ ]:

Admin_min_effective_2015_2016c = clean_admin_min_effective(Ed_Effective_2015_2016, '2015-2016')
Admin_min_effective_2016_2017c = clean_admin_min_effective(Ed_Effective_2016_2017, '2016-2017')
Admin_min_effective_2017_2018c = clean_admin_min_effective(Ed_Effective_2017_2018, '2017-2018')
Admin_min_effective_2018_2019c = clean_admin_min_effective(Ed_Effective_2018_2019, '2018-2019')
Admin_min_effective_2020_2021c = clean_admin_min_effective(Ed_Effective_2020_2021, '2020-2021')

Admin_min_effective_2015_2016cc = filter_schools(Admin_min_effective_2015_2016c, MIData2020_2021c)
Admin_min_effective_2016_2017cc = filter_schools(Admin_min_effective_2016_2017c, MIData2020_2021c)
Admin_min_effective_2017_2018cc = filter_schools(Admin_min_effective_2017_2018c, MIData2020_2021c)
Admin_min_effective_2018_2019cc = filter_schools(Admin_min_effective_2018_2019c, MIData2020_2021c)
Admin_min_effective_2020_2021cc = filter_schools(Admin_min_effective_2020_2021c, MIData2020_2021c)

In [ ]:
def clean_teacher_min_effective(df, year):
  df = df.rename(columns=lambda x: x.strip())
  df.columns = df.columns.str.lower()  
  df = df.loc[df['building name'] == 'All Buildings']
  df = df.loc[df['assignment_type'] == 'Teacher']
  df = df.loc[df['effectiveness_category'] == 'MinimallyEffective']
  df['schoolyear'] = year
  df = df.rename(columns={'district code': 'districtcode', 'district name': 'districtname', 'percentage':'teacher_min_effective_percentage'})
  df = df[['districtcode', 'teacher_min_effective_percentage']]
  df['districtcode'] = df['districtcode'].astype(str)

  # df = df.loc[df['districtname'] == 'Forest Hills Public Schools']
  return df

In [ ]:
Teacher_min_effective_2015_2016c = clean_teacher_min_effective(Ed_Effective_2015_2016, '2015-2016')
Teacher_min_effective_2016_2017c = clean_teacher_min_effective(Ed_Effective_2016_2017, '2016-2017')
Teacher_min_effective_2017_2018c = clean_teacher_min_effective(Ed_Effective_2017_2018, '2017-2018')
Teacher_min_effective_2018_2019c = clean_teacher_min_effective(Ed_Effective_2018_2019, '2018-2019')
Teacher_min_effective_2020_2021c = clean_teacher_min_effective(Ed_Effective_2020_2021, '2020-2021')

Teacher_min_effective_2015_2016cc = filter_schools(Teacher_min_effective_2015_2016c, MIData2020_2021c)
Teacher_min_effective_2016_2017cc = filter_schools(Teacher_min_effective_2016_2017c, MIData2020_2021c)
Teacher_min_effective_2017_2018cc = filter_schools(Teacher_min_effective_2017_2018c, MIData2020_2021c)
Teacher_min_effective_2018_2019cc = filter_schools(Teacher_min_effective_2018_2019c, MIData2020_2021c)
Teacher_min_effective_2020_2021cc = filter_schools(Teacher_min_effective_2020_2021c, MIData2020_2021c)

# Finance



In [ ]:
def clean_finance(df, year):
  df = df.rename(columns=lambda x: x.strip())
  df.columns = df.columns.str.lower()  

  df = df.rename(columns={'district code': 'districtcode', 'district name': 'districtname', 'fiscal year':'fiscalyear', 'fund balance as % of expenditure':'fundbalanceas%ofexpenditure'})
  df['fiscalyear'] = year
  df = df[['districtcode', 'fundbalanceas%ofexpenditure']]
  df['districtcode'] = df['districtcode'].astype('Int64')
  df['districtcode'] = df['districtcode'].astype(str)
  
  # df = df.loc[df['districtname'] == 'Forest Hills Public Schools']
  return df

In [ ]:
Finance_2015_2016c = clean_finance(Finance_2015_2016, '2015-2016')
Finance_2016_2017c = clean_finance(Finance_2016_2017, '2016-2017')
Finance_2017_2018c = clean_finance(Finance_2017_2018, '2017-2018')
Finance_2018_2019c = clean_finance(Finance_2018_2019, '2018-2019')
Finance_2020_2021c = clean_finance(Finance_2020_2021, '2020-2021')

Finance_2015_2016cc = filter_schools(Finance_2015_2016c, MIData2020_2021c)
Finance_2016_2017cc = filter_schools(Finance_2016_2017c, MIData2020_2021c)
Finance_2017_2018cc = filter_schools(Finance_2017_2018c, MIData2020_2021c)
Finance_2018_2019cc = filter_schools(Finance_2018_2019c, MIData2020_2021c)
Finance_2020_2021cc = filter_schools(Finance_2020_2021c, MIData2020_2021c)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# Demographic data from American Community Survey

In [ ]:
Demographic_Data = pd.read_csv('/drive/My Drive/SIADS_697_698_Capstone/4_Colabs/3_Data_Final/Demographic_Data_Original.csv') 
Demographic_Data = Demographic_Data.rename(columns={'SchoolDistrict': 'districtname'})

# Merge

In [ ]:
# Combine all dataframes
df2015_2016 = pd.merge(MIData2015_2016c,Attendance_2015_2016cc,on='districtcode')
df2015_2016 = pd.merge(df2015_2016,Student_2015_2016cc,on='districtcode')
df2015_2016 = pd.merge(df2015_2016,Admin_ineffective_2015_2016cc,on='districtcode')
df2015_2016 = pd.merge(df2015_2016,Admin_min_effective_2015_2016cc,on='districtcode')
df2015_2016 = pd.merge(df2015_2016,Teacher_min_effective_2015_2016cc,on='districtcode')
df2015_2016 = pd.merge(df2015_2016,Finance_2015_2016cc,on='districtcode')

df2016_2017 = pd.merge(MIData2016_2017c,Attendance_2016_2017cc,on='districtcode')
df2016_2017 = pd.merge(df2016_2017,Student_2016_2017cc,on='districtcode')
df2016_2017 = pd.merge(df2016_2017,Admin_ineffective_2016_2017cc,on='districtcode')
df2016_2017 = pd.merge(df2016_2017,Admin_min_effective_2016_2017cc,on='districtcode')
df2016_2017 = pd.merge(df2016_2017,Teacher_min_effective_2016_2017cc,on='districtcode')
df2016_2017 = pd.merge(df2016_2017,Finance_2016_2017cc,on='districtcode')

df2017_2018 = pd.merge(MIData2017_2018c,Attendance_2017_2018cc,on='districtcode')
df2017_2018 = pd.merge(df2017_2018,Student_2017_2018cc,on='districtcode')
df2017_2018 = pd.merge(df2017_2018,Admin_ineffective_2017_2018cc,on='districtcode')
df2017_2018 = pd.merge(df2017_2018,Admin_min_effective_2017_2018cc,on='districtcode')
df2017_2018 = pd.merge(df2017_2018,Teacher_min_effective_2017_2018cc,on='districtcode')
df2017_2018 = pd.merge(df2017_2018,Finance_2017_2018cc,on='districtcode')

df2018_2019 = pd.merge(MIData2018_2019c,Attendance_2018_2019cc,on='districtcode')
df2018_2019 = pd.merge(df2018_2019,Student_2018_2019cc,on='districtcode')
df2018_2019 = pd.merge(df2018_2019,Admin_ineffective_2018_2019cc,on='districtcode')
df2018_2019 = pd.merge(df2018_2019,Admin_min_effective_2018_2019cc,on='districtcode')
df2018_2019 = pd.merge(df2018_2019,Teacher_min_effective_2018_2019cc,on='districtcode')
df2018_2019 = pd.merge(df2018_2019,Finance_2018_2019cc,on='districtcode')

df2020_2021 = pd.merge(MIData2020_2021c,Attendance_2020_2021cc,on='districtcode')
df2020_2021 = pd.merge(df2020_2021,Student_2020_2021cc,on='districtcode')
df2020_2021 = pd.merge(df2020_2021,Admin_ineffective_2020_2021cc,on='districtcode')
df2020_2021 = pd.merge(df2020_2021,Admin_min_effective_2020_2021cc,on='districtcode')
df2020_2021 = pd.merge(df2020_2021,Teacher_min_effective_2020_2021cc,on='districtcode')
df2020_2021 = pd.merge(df2020_2021,Finance_2020_2021cc,on='districtcode')

Cell_Phone_DF = pd.concat([df2015_2016, df2016_2017], axis=0)
Cell_Phone_DF = pd.concat([Cell_Phone_DF, df2017_2018], axis=0)
Cell_Phone_DF = pd.concat([Cell_Phone_DF, df2018_2019], axis=0)
Cell_Phone_DF = pd.concat([Cell_Phone_DF, df2020_2021], axis=0)

# Merge academic data and demographic data
Cell_Phone_DF = pd.merge(Cell_Phone_DF ,Demographic_Data , on = 'districtname')

# Drop districts with cell phone bans besides Forest Hills Public Schools, and schools who might have a cell phone ban
Cell_Phone_DF = Cell_Phone_DF.drop(Cell_Phone_DF[Cell_Phone_DF.districtname == 'Grosse Pointe Public Schools'].index) 
Cell_Phone_DF = Cell_Phone_DF.drop(Cell_Phone_DF[Cell_Phone_DF.districtname == 'Ionia Public Schools'].index) 
Cell_Phone_DF = Cell_Phone_DF.drop(Cell_Phone_DF[Cell_Phone_DF.districtname == 'Plymouth-Canton Community Schools'].index) 
Cell_Phone_DF = Cell_Phone_DF.drop(Cell_Phone_DF[Cell_Phone_DF.districtname == 'Hopkins Public Schools'].index) 
Cell_Phone_DF = Cell_Phone_DF.drop(Cell_Phone_DF[Cell_Phone_DF.districtname == 'St. Charles Community Schools'].index) 
Cell_Phone_DF = Cell_Phone_DF.drop(Cell_Phone_DF[Cell_Phone_DF.districtname == 'Swan Valley School District'].index)
Cell_Phone_DF = Cell_Phone_DF.drop(Cell_Phone_DF[Cell_Phone_DF.districtname == 'Croswell-Lexington Community Schools'].index) 
Cell_Phone_DF = Cell_Phone_DF.drop(Cell_Phone_DF[Cell_Phone_DF.districtname == 'Capac Community Schools'].index) 
Cell_Phone_DF = Cell_Phone_DF.drop(Cell_Phone_DF[Cell_Phone_DF.districtname == 'Colon Community Schools'].index) 
Cell_Phone_DF = Cell_Phone_DF.drop(Cell_Phone_DF[Cell_Phone_DF.districtname == 'Cass City Public Schools'].index) 
Cell_Phone_DF = Cell_Phone_DF.drop(Cell_Phone_DF[Cell_Phone_DF.districtname == 'Crestwood School District'].index) 
Cell_Phone_DF = Cell_Phone_DF.drop(Cell_Phone_DF[Cell_Phone_DF.districtname == 'The School District of the City of Harper Woods'].index) 
Cell_Phone_DF = Cell_Phone_DF.drop(Cell_Phone_DF[Cell_Phone_DF.districtname == 'Onaway Area Community School District'].index) 
Cell_Phone_DF = Cell_Phone_DF.drop(Cell_Phone_DF[Cell_Phone_DF.districtname == 'Lakeview School District'].index) 

Cell_Phone_DF = Cell_Phone_DF.sort_values(by = ['districtcode', 'schoolyear'])

# Three columns have nan that should be 0: ['admin_ineffective_percentage', 'admin_min_effective_percentage', 'teacher_min_effective_percentage']
Cell_Phone_DF = Cell_Phone_DF.fillna(0)

In [ ]:
# Reduce the number of covariates
Cell_Phone_DF10 = Cell_Phone_DF[['districtcode', 'schoolyear', 'districtname', '8th_math_percentmet', 'english_language_learners_enrollment',
       'attendance', 'english_language_learners_enrollment', 'admin_min_effective_percentage', 'teacher_min_effective_percentage',
         'Estimate Median household income in the past 12 months (in 2018 inflation-adjusted dollars)', "Estimate Total Bachelor's degree", "Estimate Total Master's degree",
         'fundbalanceas%ofexpenditure', 'Estimate Total Living in household with Supplemental Security Income (SSI), cash public assistance income, or Food Stamps/SNAP in the past 12 months']]

Cell_Phone_DF5 = Cell_Phone_DF[['districtcode', 'schoolyear', 'districtname', '8th_math_percentmet',
       'attendance', 'teacher_min_effective_percentage',
         'Estimate Median household income in the past 12 months (in 2018 inflation-adjusted dollars)',
         'fundbalanceas%ofexpenditure', 'Estimate Total Living in household with Supplemental Security Income (SSI), cash public assistance income, or Food Stamps/SNAP in the past 12 months']]

In [ ]:
# Cell_Phone_DF.loc[Cell_Phone_DF['districtname'] == 'Swan Valley School District'] 
# St. Charles Community Schools.    Swan Valley School District

# Save to file

In [ ]:
Cell_Phone_DF.to_csv('/drive/My Drive/SIADS_697_698_Capstone/4_Colabs/3_Data_Final/Cell_Phone_DF.csv')
Cell_Phone_DF5.to_csv('/drive/My Drive/SIADS_697_698_Capstone/4_Colabs/3_Data_Final/Cell_Phone_DF5.csv')
Cell_Phone_DF10.to_csv('/drive/My Drive/SIADS_697_698_Capstone/4_Colabs/3_Data_Final/Cell_Phone_DF10.csv')

# Staffing Count: Teacher Specialist Degree, Teacher >= 70, Teacher Doctoral Degree

In [ ]:
def clean_techer_specialist(df, year):
  df = df.rename(columns=lambda x: x.strip())
  df.columns = df.columns.str.lower()  
  # df = df.loc[df['building name'] == 'All Buildings']
  # df = df.loc[df['assignment_type'] == 'Teacher']
  # df = df.loc[df['effectiveness_category'] == 'MinimallyEffective']
  # df['schoolyear'] = year
  # df = df.rename(columns={'percentage':'teacher_min_effective_percentage'})
  # df = df[['schoolyear', 'districtcode', 'districtname', 'teacher_min_effective_percentage']]

  df = df.loc[df['districtname'] == 'Forest Hills Public Schools']
  return df

In [ ]:
teacher_specialist_2015_2016c = clean_techer_specialist(Staff_2015_2016, '2015-2016')
teacher_specialist_2015_2016c


,schoolyear,isdcode,isdname,districtcode,districtname,buildingcode,buildingname,countycode,countyname,entitytype,schoollevel,locale,staffgroup,reportcategoryoverall,reportcategory,headcount,headcountpercent,fte,ftepercent
41904,15 - 16 School Year,41,Kent ISD,41110,Forest Hills Public Schools,0,All Buildings in District,41.0,Kent,LEA District,Elem thru High School,Suburb: Large,Teachers,Gender,Male,154.0,24.9,152.61,25.8
41905,15 - 16 School Year,41,Kent ISD,41110,Forest Hills Public Schools,0,All Buildings in District,41.0,Kent,LEA District,Elem thru High School,Suburb: Large,Non-Instructional Paraprofessionals/Aides,Race/Ethnicity,Two or More Races,10.0,1.1,7.87,2.2
41906,15 - 16 School Year,41,Kent ISD,41110,Forest Hills Public Schools,0,All Buildings in District,41.0,Kent,LEA District,Elem thru High School,Suburb: Large,Teachers,Race/Ethnicity,American Indian or Alaska Native,2.0,0.3,1.16,0.2
41907,15 - 16 School Year,41,Kent ISD,41110,Forest Hills Public Schools,0,All Buildings in District,41.0,Kent,LEA District,Elem thru High School,Suburb: Large,Day-to-Day Paraprofessionals/Aides,Race/Ethnicity,Asian,2.0,3.6,NaN,NaN
41908,15 - 16 School Year,41,Kent ISD,41110,Forest Hills Public Schools,0,All Buildings in District,41.0,Kent,LEA District,Elem thru High School,Suburb: Large,Paraprofessionals/Aides,Race/Ethnicity,"Black, not of Hispanic origin",4.0,2.0,3.24,2.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42411,15 - 16 School Year,41,Kent ISD,41110,Forest Hills Public Schools,9712,Forest Hills Transition Center,41.0,Kent,LEA Unique Education Provider,Other,Suburb: Large,Teachers,Gender,Male,1.0,16.7,0.05,1.7
42412,15 - 16 School Year,41,Kent ISD,41110,Forest Hills Public Schools,9712,Forest Hills Transition Center,41.0,Kent,LEA Unique Education Provider,Other,Suburb: Large,Paraprofessionals/Aides,Race/Ethnicity,"White, not of Hispanic origin",2.0,100.0,2.00,100.0
42413,15 - 16 School Year,41,Kent ISD,41110,Forest Hills Public Schools,9712,Forest Hills Transition Center,41.0,Kent,LEA Unique Education Provider,Other,Suburb: Large,Paraprofessionals/Aides,All Staff,All Staff,2.0,25.0,2.00,40.0
42414,15 - 16 School Year,41,Kent ISD,41110,Forest Hills Public Schools,9712,Forest Hills Transition Center,41.0,Kent,LEA Unique Education Provider,Other,Suburb: Large,All,All Staff,All Staff,8.0,100.0,5.00,100.0


# Experiments

In [ ]:
# Save cleaned dataframes
# MIData2015_2016.to_csv('/drive/My Drive/SIADS_697_698_Capstone/3_Colabs/2_Michigan_Academic_Data_from_MI_School_Data/2_Clean_Data/Grade_8_Cleaned_2015_2016.csv')
# MIData2016_2017.to_csv('/drive/My Drive/SIADS_697_698_Capstone/3_Colabs/2_Michigan_Academic_Data_from_MI_School_Data/2_Clean_Data/Grade_8_Cleaned_2016_2017.csv')
# MIData2017_2018.to_csv('/drive/My Drive/SIADS_697_698_Capstone/3_Colabs/2_Michigan_Academic_Data_from_MI_School_Data/2_Clean_Data/Grade_8_Cleaned_2017_2018.csv')
# MIData2018_2019.to_csv('/drive/My Drive/SIADS_697_698_Capstone/3_Colabs/2_Michigan_Academic_Data_from_MI_School_Data/2_Clean_Data/Grade_8_Cleaned_2018_2019.csv')
# MIData2020_2021.to_csv('/drive/My Drive/SIADS_697_698_Capstone/3_Colabs/2_Michigan_Academic_Data_from_MI_School_Data/2_Clean_Data/Grade_8_Cleaned_2020_2021.csv')

In [ ]:
# Make sure 12 schools with cell phone bans are in master list of 413 schools with scores every year
Schools_With_Bans = [9315, 1264, 6762, 5827, 424, 2937, 3031, 2269, 3079, 2647, 4849, 3078]
Schools_With_Bans_df = MIData_Final.loc[MIData_Final['buildingcode'].isin(Schools_With_Bans)]

NameError: ignored

In [ ]:
# counter = 0
# for i in range(len(MIData_Final)):
#   df = MIData_Final.iloc[counter:counter+100]
#   counter+=100
#   df = pd.melt(df, id_vars='buildingname', value_vars=['percentmet2015_2016', 'percentmet2016_2017', 'percentmet2017_2018', 'percentmet2018_2019', 'percentmet2020_2021'])
#   import altair as alt
#   alt.Chart(df).mark_line().encode( x='variable',
#     y='value',
#     color='buildingname' ).properties( width=800, height=600).show()


In [ ]:
for i in range(len(MIData_Final)):
  df = MIData_Final.iloc[i:i+1]
  df_ban = Schools_With_Bans_df.iloc[11:12]
  df = pd.concat([df, df_ban], axis = 0)
  df = pd.melt(df, id_vars='buildingname', value_vars=['percentmet2015_2016', 'percentmet2016_2017', 'percentmet2017_2018', 'percentmet2018_2019', 'percentmet2020_2021'])
  import altair as alt
  chart = alt.Chart(df).mark_line().encode( x='variable',
    y=alt.Y('value', sort='-y'),
    color='buildingname' ).properties( width=200, height=100)
  chart.display()